In [64]:
import requests   # best library to manage HTTP transactions
from bs4 import BeautifulSoup # web-scraping library
import json

acceptMediaType = 'text/html'
userAgentHeader = 'BaskaufScraper/0.1 (steve.baskauf@vanderbilt.edu)'
requestHeaderDictionary = {
    'Accept' : acceptMediaType,
    'User-Agent': userAgentHeader
}
url = 'https://as.vanderbilt.edu/biosci/people/index.php?group=primary-training-faculty'
response = requests.get(url, headers = requestHeaderDictionary)
soupObject = BeautifulSoup(response.text,features="html5lib")

In [65]:
tableObject = soupObject.find_all('table')[0]
#print(tableObject)

In [66]:
rowObjectsList = tableObject.find_all('tr')
#print(rowObjectsList)

In [67]:
rowObject = rowObjectsList[21]
cellObjectsList = rowObject.find_all('td')
print(cellObjectsList)

[<td width="100"><a class="pic" href="/biosci/bio/laurence-zwiebel"><img class="img-rounded" src="/biosci/images/people/image.php/laurence-zwiebel.jpg?width=100&amp;image=/biosci/images/people/laurence-zwiebel.jpg"/></a></td>, <td align="left" class="biolink"><strong><a href="/biosci/bio/laurence-zwiebel">Laurence J. Zwiebel, Ph.D.</a></strong><br/>Cornelius Vanderbilt Chair in Biological Sciences<br/>Professor of Biological Sciences, Professor of Pharmacology</td>, <td class="contactinfo" width="25%"><a href="mailto:l.zwiebel@vanderbilt.edu">Email</a><br/><a href="https://lamp.vanderbilt.edu/zwiebellab/Home.html">Website</a><br/>615-343-1894<br/>6260 MRB III</td>]


In [68]:
nameCell = cellObjectsList[1]
print(nameCell)
name = nameCell('strong')[0].text
print(name)

<td align="left" class="biolink"><strong><a href="/biosci/bio/laurence-zwiebel">Laurence J. Zwiebel, Ph.D.</a></strong><br/>Cornelius Vanderbilt Chair in Biological Sciences<br/>Professor of Biological Sciences, Professor of Pharmacology</td>
Laurence J. Zwiebel, Ph.D.


In [69]:
dirtyText  = str(nameCell)
# get rid of trailing td tag
nameCellText = dirtyText.split('</td>')[0]
cellLines = nameCellText.split('<br/>')
roles = []
for lineIndex in range(1, len(cellLines)):
    roleDict = {}
    print(cellLines[lineIndex])
    if ' of ' in cellLines[lineIndex]:
        pieces = cellLines[lineIndex].split(' of ')
        roleDict['title'] = pieces[0]
        roleDict['department'] = pieces[1]
        roles.append(roleDict)
    elif ' in ' in cellLines[lineIndex]:
        pieces = cellLines[lineIndex].split(' in ')
        roleDict['title'] = pieces[0]
        roleDict['department'] = pieces[1]
        roles.append(roleDict)
    else:
        roleDict['title'] = cellLines[lineIndex]
        roleDict['department'] = ''
        roles.append(roleDict)     
    if ', Emeritus' in roleDict['department']:
        roleDict['department'] = roleDict['department'].split(', Emeritus')[0]
        roleDict['title'] = 'Emeritus ' + roleDict['title']
rolesJson = json.dumps(roles)
print(rolesJson)

Cornelius Vanderbilt Chair in Biological Sciences
Professor of Biological Sciences, Professor of Pharmacology
[{'title': 'Cornelius Vanderbilt Chair', 'department': 'Biological Sciences'}, {'title': 'Professor', 'department': 'Biological Sciences, Professor'}]
[{"title": "Cornelius Vanderbilt Chair", "department": "Biological Sciences"}, {"title": "Professor", "department": "Biological Sciences, Professor"}]


In [71]:
import requests   # best library to manage HTTP transactions
from bs4 import BeautifulSoup # web-scraping library
import csv
import json

def writeCsv(fileName, array):
    fileObject = open(fileName, 'w', newline='', encoding='utf-8')
    writerObject = csv.writer(fileObject)
    for row in array:
        writerObject.writerow(row)
    fileObject.close()

acceptMediaType = 'text/html'
userAgentHeader = 'BaskaufScraper/0.1 (steve.baskauf@vanderbilt.edu)'
requestHeaderDictionary = {
    'Accept' : acceptMediaType,
    'User-Agent': userAgentHeader
}

outputTable = [['name', 'degree', 'role', 'category']]
categories = ['primary-training-faculty', 'research-and-teaching-faculty', 'secondary-faculty', 'postdoc-fellows', 'emeriti']
degreeList = [
    {'string': 'Ph.D.', 'value': 'Ph.D.'},
    {'string': 'PhD', 'value': 'Ph.D.'},
    {'string': 'D.Phil.', 'value': 'D.Phil.'},
    {'string': 'J.D.', 'value': 'J.D.'}
     ]

for category in categories:
    url = 'https://as.vanderbilt.edu/biosci/people/index.php?group=' + category
    response = requests.get(url, headers = requestHeaderDictionary)
    soupObject = BeautifulSoup(response.text,features="html5lib")

    # get the first table from the page
    tableObject = soupObject.find_all('table')[0]
    
    # get the rows from the table
    rowObjectsList = tableObject.find_all('tr')
    for rowObject in rowObjectsList:
        # get the cells from each row
        cellObjectsList = rowObject.find_all('td')
        # picture is in cell 0, name and title is in cell 1
        nameCell = cellObjectsList[1]
        # the name part is bolded
        name = nameCell('strong')[0].text
        
        # separate degrees from names
        degree = ''
        for testDegree in degreeList:
            if testDegree['string'] in name:
                name = name.partition(', ' + testDegree['string'])[0]
                # correct any malformed strings
                degree = testDegree['value']

        try:
            # process the roles text
            dirtyText  = str(nameCell)
            # get rid of trailing td tag
            nameCellText = dirtyText.split('</td>')[0]
            cellLines = nameCellText.split('<br/>')
            roles = []
            for lineIndex in range(1, len(cellLines)):
                roleDict = {}
                if ' of ' in cellLines[lineIndex]:
                    pieces = cellLines[lineIndex].split(' of ')
                    roleDict['title'] = pieces[0]
                    roleDict['department'] = pieces[1]
                    roles.append(roleDict)
                elif ' in ' in cellLines[lineIndex]:
                    pieces = cellLines[lineIndex].split(' in ')
                    roleDict['title'] = pieces[0]
                    roleDict['department'] = pieces[1]
                    roles.append(roleDict)
                else:
                    roleDict['title'] = cellLines[lineIndex]
                    roleDict['department'] = ''
                    roles.append(roleDict)
                if ', Emeritus' in roleDict['department']:
                    roleDict['department'] = roleDict['department'].split(', Emeritus')[0]
                    roleDict['title'] = 'Emeritus ' + roleDict['title']
            rolesJson = json.dumps(roles)

        except:
            rolesJson = ''
        outputTable.append([name, degree, rolesJson, category])            

fileName = 'bsci-employees.csv'
writeCsv(fileName, outputTable)
print('done')

done
